In [11]:
import pandas as pd
import numpy as np

import autorootcwd  # noqa
from hamilton import driver


from src.data import data_pipeline
from src.data.pydantic_models import BearingDataset
from functools import partial
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import (
    RandomizedSearchCV,
    GroupKFold,
    cross_validate,
    cross_val_predict,
)
from scipy.stats import randint, loguniform
import logging
import random
from itertools import combinations, product
from src.utils.add_signal_data import add_signal_data_ottawa, add_signal_data_cwru

In [12]:
def get_cv_splits(
    df,
    n_bearings_per_fault_mode=2,
    run=0,
    random_state=42,
):
    """
    Generate custom CV splits where each test set contains 2 IDs from each fault type.
    """

    # Get unique IDs per fault type
    inner_ids = [1, 2, 3, 4, 5]
    outer_ids = [6, 7, 8, 9, 10]
    ball_ids = [11, 12, 13, 14, 15]
    cage_ids = [16, 17, 18, 19, 20]

    # All 2-combinations for each
    inner_combos = list(combinations(inner_ids, n_bearings_per_fault_mode))
    outer_combos = list(combinations(outer_ids, n_bearings_per_fault_mode))
    ball_combos = list(combinations(ball_ids, n_bearings_per_fault_mode))
    cage_combos = list(combinations(cage_ids, n_bearings_per_fault_mode))

    # Cartesian product to form full test sets (can be huge!)
    all_combos = list(product(inner_combos, outer_combos, ball_combos, cage_combos))

    # Shuffle
    rng = random.Random(random_state)
    rng.shuffle(all_combos)

    # Limit to n_splits
    selected_combos = all_combos[run]
    print(f"Selected test IDs: {selected_combos}")

    inner_test, outer_test, ball_test, cage_test = selected_combos
    # Combine all test IDs into one set
    test_ids = set(inner_test) | set(outer_test) | set(ball_test) | set(cage_test)
    test_mask = df["bearing_id"].astype(int).isin(test_ids)
    test_idx = df[test_mask].index.values.tolist()
    train_idx = df[~test_mask].index.values.tolist()
   
    cv = (train_idx, test_idx)

    return cv

In [13]:
metadata = pd.read_pickle("/data/bearing_datasets/ottawa/processed/files_metadata.bz2")
features = pd.read_pickle("data/features/ottawa_features_segmented.pkl")

In [14]:
df = features.merge(metadata, on="waveform_id", how="left").reset_index(drop=True)

In [15]:
df.bearing_id.unique()

array(['16', '2', '17', '8', '1', '5', '15', '19', '9', '11', '3', '20',
       '10', '6', '12', '13', '18', '4', '14', '7'], dtype=object)

In [16]:
#df = df.drop_duplicates(subset='waveform_id').reset_index(drop=True)

In [17]:
val_cvs = [get_cv_splits(df=df, run=run)
 for run in range(5)]

Selected test IDs: ((1, 5), (8, 9), (13, 14), (16, 18))
Selected test IDs: ((2, 5), (7, 10), (13, 14), (16, 19))
Selected test IDs: ((3, 4), (6, 9), (12, 15), (16, 18))
Selected test IDs: ((1, 2), (8, 9), (12, 15), (16, 17))
Selected test IDs: ((1, 5), (8, 9), (13, 14), (19, 20))


In [18]:
features_list = ['acceleration/rms/global', 'acceleration/pk-pk/global',
       'acceleration/kurt/global', 'acceleration/skewness/global',
       'acceleration/fc/global',
       'envelope/spectralPeak/1.0x-bpfo/500-10000',
       'envelope/spectralPeak/2.0x-bpfo/500-10000',
       'envelope/spectralPeak/3.0x-bpfo/500-10000',
       'envelope/spectralPeak/4.0x-bpfo/500-10000',
       'envelope/spectralPeak/5.0x-bpfo/500-10000',
       'envelope/spectralPeak/1.0x-bpfi/500-10000',
       'envelope/spectralPeak/2.0x-bpfi/500-10000',
       'envelope/spectralPeak/3.0x-bpfi/500-10000',
       'envelope/spectralPeak/4.0x-bpfi/500-10000',
       'envelope/spectralPeak/5.0x-bpfi/500-10000',
       'envelope/spectralPeak/1.0x-bsf/500-10000',
       'envelope/spectralPeak/2.0x-bsf/500-10000',
       'envelope/spectralPeak/3.0x-bsf/500-10000',
       'envelope/spectralPeak/4.0x-bsf/500-10000',
       'envelope/spectralPeak/5.0x-bsf/500-10000',
       'envelope/spectralPeak/1.0x-ftf/500-10000',
       'envelope/spectralPeak/2.0x-ftf/500-10000',
       'envelope/spectralPeak/3.0x-ftf/500-10000',
       'envelope/spectralPeak/4.0x-ftf/500-10000',
       'envelope/spectralPeak/5.0x-ftf/500-10000']

In [23]:
X = df[features_list].copy()
y = df[['inner', 'outer', 'ball', 'cage']].copy()

# Features (segmented)

## RF

In [31]:
random_search = RandomizedSearchCV(
    estimator=MultiOutputClassifier(RandomForestClassifier()),
    param_distributions={
        "estimator__n_estimators": [200],
        "estimator__max_features": ["sqrt", "log2"],
        "estimator__criterion": ["gini", "entropy", "log_loss"],
        "estimator__max_depth": randint(low=2, high=60),
        "estimator__min_samples_split": randint(low=2, high=20),
        "estimator__min_samples_leaf": randint(low=1, high=20),
        "estimator__ccp_alpha": loguniform(1e-5, 1),
    },
    n_iter=250,
    cv=val_cvs,
    scoring="roc_auc",
    verbose=2,
    n_jobs=-1,
)

random_search.fit(X, y)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV] END estimator__ccp_alpha=0.0010202402883986654, estimator__criterion=log_loss, estimator__max_depth=8, estimator__max_features=log2, estimator__min_samples_leaf=11, estimator__min_samples_split=2, estimator__n_estimators=200; total time=   1.1s
[CV] END estimator__ccp_alpha=0.016263441148103557, estimator__criterion=entropy, estimator__max_depth=26, estimator__max_features=log2, estimator__min_samples_leaf=15, estimator__min_samples_split=12, estimator__n_estimators=200; total time=   1.1s
[CV] END estimator__ccp_alpha=0.02279967799370947, estimator__criterion=gini, estimator__max_depth=59, estimator__max_features=sqrt, estimator__min_samples_leaf=5, estimator__min_samples_split=13, estimator__n_estimators=200; total time=   1.1s
[CV] END estimator__ccp_alpha=0.0010202402883986654, estimator__criterion=log_loss, estimator__max_depth=8, estimator__max_features=log2, estimator__min_samples_leaf=11, estimator__min_sample

RandomizedSearchCV(cv=[([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                         24, 25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 65, 66, 67,
                         68, 69, ...],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 30, 31, 32, 33, 34, 35,
                         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, ...]),
                       ([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
                         48, 49, ...],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                         16, 17, 1...
                                        'estimator__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa43cd8b7d0>,
                                        'estimator__max_features': ['sqrt',
                                                                    'log2'],
                                        'estimator__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa43cd88a50>,
                                        'estimator__min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa43cd88110>,
                                        'estimator__n_estimators': [200]},
                   scoring='roc_auc', verbose=2)

In [32]:
results = pd.DataFrame(random_search.cv_results_).sort_values(by="rank_test_score")

In [33]:
results.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__ccp_alpha,param_estimator__criterion,param_estimator__max_depth,param_estimator__max_features,param_estimator__min_samples_leaf,param_estimator__min_samples_split,param_estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
90,1.161279,0.067809,0.051735,0.001119,0.075020,entropy,30,sqrt,8,17,200,"{'estimator__ccp_alpha': 0.0750201477631042, '...",0.935438,0.911219,0.874547,0.760250,0.857594,0.867809,0.060294,1
89,1.086177,0.016604,0.051910,0.000799,0.011634,entropy,39,sqrt,16,7,200,"{'estimator__ccp_alpha': 0.011634332494909434,...",0.937937,0.915531,0.875062,0.747203,0.860172,0.867181,0.066106,2
20,1.074565,0.009326,0.053154,0.000345,0.000703,entropy,4,sqrt,16,9,200,{'estimator__ccp_alpha': 0.0007028873842496256...,0.938406,0.907156,0.867781,0.762172,0.858406,0.866784,0.059623,3
5,1.069900,0.011701,0.052939,0.001662,0.000076,log_loss,48,log2,9,18,200,{'estimator__ccp_alpha': 7.593241700922166e-05...,0.934531,0.915938,0.871688,0.751750,0.857219,0.866225,0.063801,4
146,1.074608,0.011672,0.051399,0.000589,0.000276,entropy,53,sqrt,19,10,200,{'estimator__ccp_alpha': 0.0002761188687479064...,0.931563,0.909000,0.875641,0.752094,0.862062,0.866072,0.062017,5


In [34]:
results.loc[22]

mean_fit_time                                                                  1.050319
std_fit_time                                                                   0.012528
mean_score_time                                                                0.053192
std_score_time                                                                 0.000189
param_estimator__ccp_alpha                                                     0.000094
param_estimator__criterion                                                      entropy
param_estimator__max_depth                                                           10
param_estimator__max_features                                                      log2
param_estimator__min_samples_leaf                                                    16
param_estimator__min_samples_split                                                   14
param_estimator__n_estimators                                                       200
params                          

In [35]:
best_params = results.loc[22]["params"]

In [36]:
best_params.values()

dict_values([np.float64(9.381173398025799e-05), 'entropy', 10, 'log2', 16, 14, 200])

In [37]:
keys = []
for i in best_params.keys():
    keys.append(i.replace("estimator__", ""))

In [38]:
best_params = {k: i for (k,i) in zip(keys, best_params.values())}

In [39]:
best_params

{'ccp_alpha': np.float64(9.381173398025799e-05),
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_leaf': 16,
 'min_samples_split': 14,
 'n_estimators': 200}

In [40]:
from sklearn.metrics import roc_auc_score

#### WITHOUT COMBINEDS
cvs = [get_cv_splits(df=df, run=run) for run in range(5,105)]

aucs = []
for cv in cvs:

    X_train = X.iloc[cv[0]]
    X_test = X.iloc[cv[1]]

    y_train = y.iloc[cv[0]]
    y_test = y.iloc[cv[1]]

    model = MultiOutputClassifier(RandomForestClassifier(random_state=42, **best_params), n_jobs=-1)

    model.fit(X_train, y_train)
    y_probas = model.predict_proba(X_test)
    proba_inner = y_probas[0][:, 1]
    proba_outer = y_probas[1][:, 1]
    proba_ball = y_probas[2][:, 1]
    proba_cage = y_probas[3][:, 1]

    auroc_outer = roc_auc_score(y_test["outer"], proba_outer)
    auroc_inner = roc_auc_score(y_test["inner"], proba_inner)
    auroc_ball = roc_auc_score(y_test["ball"], proba_ball)
    auroc_cage = roc_auc_score(y_test["cage"], proba_cage)

    macro_auc = np.mean([auroc_outer, auroc_inner, auroc_ball, auroc_cage])
    print(f"Macro AUC: {macro_auc:.4f} | Outer AUC: {auroc_outer:.4f} | Inner AUC: {auroc_inner:.4f} | Ball AUC: {auroc_ball:.4f} | Cage AUC: {auroc_cage:.4f}")
    aucs.append([macro_auc, auroc_outer, auroc_inner, auroc_ball, auroc_cage])


Selected test IDs: ((1, 3), (8, 9), (13, 14), (16, 19))
Selected test IDs: ((3, 4), (7, 9), (11, 12), (19, 20))
Selected test IDs: ((1, 4), (7, 10), (13, 14), (19, 20))
Selected test IDs: ((1, 4), (6, 10), (11, 12), (17, 19))
Selected test IDs: ((4, 5), (6, 9), (11, 13), (17, 19))
Selected test IDs: ((1, 5), (7, 10), (11, 14), (17, 19))
Selected test IDs: ((2, 3), (6, 9), (11, 15), (17, 20))
Selected test IDs: ((4, 5), (6, 7), (11, 12), (17, 19))
Selected test IDs: ((2, 4), (9, 10), (11, 14), (17, 18))
Selected test IDs: ((1, 5), (6, 7), (14, 15), (16, 20))
Selected test IDs: ((3, 4), (7, 9), (14, 15), (16, 17))
Selected test IDs: ((2, 3), (8, 9), (11, 15), (17, 20))
Selected test IDs: ((2, 3), (6, 10), (12, 15), (17, 20))
Selected test IDs: ((2, 3), (6, 7), (12, 14), (17, 20))
Selected test IDs: ((2, 3), (6, 8), (13, 14), (18, 20))
Selected test IDs: ((1, 3), (6, 9), (13, 15), (18, 19))
Selected test IDs: ((3, 5), (6, 10), (12, 13), (16, 20))
Selected test IDs: ((2, 3), (7, 10), (11, 

In [41]:
macro_aucs = [i[0] for i in aucs]
np.mean(macro_aucs), np.std(macro_aucs)

(np.float64(0.8557956250000001), np.float64(0.047383260878958486))

## SVM

In [42]:
from sklearn.svm import SVC

random_search = RandomizedSearchCV(
    estimator=MultiOutputClassifier(SVC(probability=True)),
    param_distributions={
        "estimator__C": loguniform(1e-3, 1e3),
        "estimator__gamma": ["scale", "auto"],
        "estimator__kernel": ["rbf"],
    },
    n_iter=250,
    cv=val_cvs,
    scoring="roc_auc",
    verbose=2,
    n_jobs=-1,
)

random_search.fit(X, y)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV] END estimator__C=545.9914847144264, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=545.9914847144264, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=545.9914847144264, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=545.9914847144264, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=0.029935224762834133, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=0.029935224762834133, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=0.029935224762834133, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=0.029935224762834133, estimator__gamma=scale, estimator__kernel=rbf; total time=   0.1s
[CV] END estimator__C=0.029935224762834133, estimator__gamma=scale, 

RandomizedSearchCV(cv=[([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                         24, 25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 65, 66, 67,
                         68, 69, ...],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 30, 31, 32, 33, 34, 35,
                         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, ...]),
                       ([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
                         48, 49, ...],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                         16, 17, 1...
                        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
                         44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
                         58, 59, ...])],
                   estimator=MultiOutputClassifier(estimator=SVC(probability=True)),
                   n_iter=250, n_jobs=-1,
                   param_distributions={'estimator__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa43e39a650>,
                                        'estimator__gamma': ['scale', 'auto'],
                                        'estimator__kernel': ['rbf']},
                   scoring='roc_auc', verbose=2)

In [43]:
results = pd.DataFrame(random_search.cv_results_).sort_values(by="rank_test_score")

In [44]:
results.iloc[0]

mean_fit_time                                                       0.049028
std_fit_time                                                         0.00285
mean_score_time                                                     0.008318
std_score_time                                                      0.000243
param_estimator__C                                                480.689254
param_estimator__gamma                                                 scale
param_estimator__kernel                                                  rbf
params                     {'estimator__C': 480.6892544768522, 'estimator...
split0_test_score                                                   0.928656
split1_test_score                                                   0.795359
split2_test_score                                                   0.858531
split3_test_score                                                   0.627938
split4_test_score                                                     0.9225

In [45]:
best_params = results.iloc[0]["params"]

In [46]:
best_params.values()

dict_values([np.float64(480.6892544768522), 'scale', 'rbf'])

In [47]:
keys = []
for i in best_params.keys():
    keys.append(i.replace("estimator__", ""))

In [48]:
best_params = {k: i for (k,i) in zip(keys, best_params.values())}

In [49]:
best_params

{'C': np.float64(480.6892544768522), 'gamma': 'scale', 'kernel': 'rbf'}

In [51]:
from sklearn.metrics import roc_auc_score

aucs = []
for cv in cvs:

    X_train = X.iloc[cv[0]]
    X_test = X.iloc[cv[1]]

    y_train = y.iloc[cv[0]]
    y_test = y.iloc[cv[1]]

    model = MultiOutputClassifier(SVC(random_state=42, **best_params, probability=True), n_jobs=-1)

    model.fit(X_train, y_train)
    y_probas = model.predict_proba(X_test)
    proba_outer = y_probas[1][:, 1]
    proba_inner = y_probas[0][:, 1]
    proba_ball = y_probas[2][:, 1]
    proba_cage = y_probas[3][:, 1]

    auroc_outer = roc_auc_score(y_test["outer"], proba_outer)
    auroc_inner = roc_auc_score(y_test["inner"], proba_inner)
    auroc_ball = roc_auc_score(y_test["ball"], proba_ball)
    auroc_cage = roc_auc_score(y_test["cage"], proba_cage)

    macro_auc = np.mean([auroc_outer, auroc_inner, auroc_ball, auroc_cage])
    print(f"Macro AUC: {macro_auc:.4f} | Outer AUC: {auroc_outer:.4f} | Inner AUC: {auroc_inner:.4f} | Ball AUC: {auroc_ball:.4f} | Cage AUC: {auroc_cage:.4f}")
    aucs.append([macro_auc, auroc_outer, auroc_inner, auroc_ball, auroc_cage])


Macro AUC: 0.9156 | Outer AUC: 0.9096 | Inner AUC: 0.9803 | Ball AUC: 0.8506 | Cage AUC: 0.9219
Macro AUC: 0.8488 | Outer AUC: 0.5505 | Inner AUC: 1.0000 | Ball AUC: 0.8809 | Cage AUC: 0.9637
Macro AUC: 0.9190 | Outer AUC: 0.8869 | Inner AUC: 0.9969 | Ball AUC: 0.8036 | Cage AUC: 0.9888
Macro AUC: 0.9127 | Outer AUC: 0.9842 | Inner AUC: 0.9434 | Ball AUC: 0.9810 | Cage AUC: 0.7421
Macro AUC: 0.8246 | Outer AUC: 0.9661 | Inner AUC: 0.9601 | Ball AUC: 0.6008 | Cage AUC: 0.7714
Macro AUC: 0.8413 | Outer AUC: 0.8955 | Inner AUC: 0.9280 | Ball AUC: 0.7496 | Cage AUC: 0.7923
Macro AUC: 0.8223 | Outer AUC: 0.9990 | Inner AUC: 0.7864 | Ball AUC: 0.8440 | Cage AUC: 0.6599
Macro AUC: 0.8067 | Outer AUC: 0.7812 | Inner AUC: 0.9481 | Ball AUC: 0.7526 | Cage AUC: 0.7448
Macro AUC: 0.8742 | Outer AUC: 0.9523 | Inner AUC: 0.9483 | Ball AUC: 0.7596 | Cage AUC: 0.8367
Macro AUC: 0.8487 | Outer AUC: 0.7602 | Inner AUC: 0.9832 | Ball AUC: 0.7393 | Cage AUC: 0.9121
Macro AUC: 0.6352 | Outer AUC: 0.6386 | 

In [52]:
macro_aucs = [i[0] for i in aucs]
np.mean(macro_aucs), np.std(macro_aucs)

(np.float64(0.8133223437500001), np.float64(0.08607596279093102))